In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

# Machine Learning Experimentation Project

I will be using the lessons I learned in class and self study to experiment with a machine learning dataset and model.

### Project Information
This project uses the EuroSAT dataset from tensorflow https://www.tensorflow.org/datasets/catalog/eurosat.
EuroSAT is a dataset of 27000 satellite images of different geological terrains (rivers, forest, residential areas, etc.).
The goal of this project is to make a model which can most accurately place an image into one of 10 categories.

## Helper Function
Just a function to help with mapping

In [2]:
def normalize_img(datapoint):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(datapoint['image'], tf.float32) / 255., datapoint['label']

## Test 1 (Simple DNN)
The first test will be a bare bones DNN with only 3 layers.

In [3]:
dataset = tfds.builder('eurosat')
info = dataset.info # documentation for the dataset
print(info)

tfds.core.DatasetInfo(
    name='eurosat',
    full_name='eurosat/rgb/2.0.0',
    description="""
    EuroSAT dataset is based on Sentinel-2 satellite images covering 13 spectral
    bands and consisting of 10 classes with 27000 labeled and
    geo-referenced samples.
    
    Two datasets are offered:
    - rgb: Contains only the optical R, G, B frequency bands encoded as JPEG image.
    - all: Contains all 13 bands in the original value range (float32).
    
    URL: https://github.com/phelber/eurosat
    """,
    config_description="""
    Sentinel-2 RGB channels
    """,
    homepage='https://github.com/phelber/eurosat',
    data_dir='C:\\Users\\dargu\\tensorflow_datasets\\eurosat\\rgb\\2.0.0',
    file_format=tfrecord,
    download_size=89.91 MiB,
    dataset_size=89.50 MiB,
    features=FeaturesDict({
        'filename': Text(shape=(), dtype=string),
        'image': Image(shape=(64, 64, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),

In [4]:
print(info.features)
print("These are the attributes that can be accessed from each training element\n")

print("Classes:")
for i in range(info.features["label"].num_classes):
    print('\t',info.features["label"].int2str(i))

FeaturesDict({
    'filename': Text(shape=(), dtype=string),
    'image': Image(shape=(64, 64, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
})
These are the attributes that can be accessed from each training element

Classes:
	 AnnualCrop
	 Forest
	 HerbaceousVegetation
	 Highway
	 Industrial
	 Pasture
	 PermanentCrop
	 Residential
	 River
	 SeaLake


In [5]:
print("Using 90% data for training and rest 10% for testing")
(train, test) = tfds.load("eurosat/rgb", split=["train[:90%]", "train[90%:]"])
print('Train:', len(train), 'Test:', len(test))

Using 90% data for training and rest 10% for testing
Train: 24300 Test: 2700


In [6]:
print("Batching the training set with a size of 128.")
print("This means the model can work on 128 images at once. train.batch()\n")
train = train.batch(128)
print("Normalizing images so pixel values are between 0 and 1. train.map()")
train = train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
print("Storing training set in cache so subsequent calls are faster. train.cache()")
train = train.cache() # stores training set in cache so subsequent calls are faster
print("Shuffling training set. train.shuffle()\n")
train = train.shuffle(len(train)) # shuffles set

print("Batching, mapping, and caching test set as well")
test = test.batch(128)
test = test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test = test.cache()

Batching the training set with a size of 128.
This means the model can work on 128 images at once. train.batch()

Normalizing images so pixel values are between 0 and 1. train.map()
Storing training set in cache so subsequent calls are faster. train.cache()
Shuffling training set. train.shuffle()

Batching, mapping, and caching test set as well


In [7]:
print("Making a model with only 3 layers. One Flatten layer and two Dense.")
print("Using relu and softmax\n")
model = Sequential([
  Flatten(input_shape=(64, 64, 3)),
  Dense(128, activation='relu'),
  Dense(info.features["label"].num_classes, activation='softmax')
])

print("Using SparseCategoricalCrossentropy for loss and metrics \n")
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=[SparseCategoricalAccuracy()]
)

print("Fitting model with 100 epochs")
model.fit(
    train,
    epochs=100,
)

# for some reason it doesn't work sometimes. In my final lookover at about 11:30, this cell is giving me errors.
# All I can say is it worked before, but isn't working now.

Making a model with only 3 layers. One Flatten layer and two Dense.
Using relu and softmax

Using SparseCategoricalCrossentropy for loss and metrics 

Fitting model with 100 epochs
Epoch 1/100
190/190 [==============================] - 3s 13ms/step - loss: 2.8138 - sparse_categorical_accuracy: 0.1975
Epoch 2/100
190/190 [==============================] - 3s 14ms/step - loss: 1.8679 - sparse_categorical_accuracy: 0.2284
Epoch 3/100
190/190 [==============================] - 3s 14ms/step - loss: 1.8288 - sparse_categorical_accuracy: 0.2463
Epoch 4/100
190/190 [==============================] - 3s 14ms/step - loss: 1.8139 - sparse_categorical_accuracy: 0.2605
Epoch 5/100
190/190 [==============================] - 3s 14ms/step - loss: 1.8011 - sparse_categorical_accuracy: 0.2649
Epoch 6/100
190/190 [==============================] - 2s 13ms/step - loss: 1.7908 - sparse_categorical_accuracy: 0.2704
Epoch 7/100
190/190 [==============================] - 2s 13ms/step - loss: 1.7817 - sparse_c

190/190 [==============================] - 2s 13ms/step - loss: 1.6818 - sparse_categorical_accuracy: 0.3326
Epoch 67/100
190/190 [==============================] - 2s 13ms/step - loss: 1.6827 - sparse_categorical_accuracy: 0.3304
Epoch 68/100
190/190 [==============================] - 2s 13ms/step - loss: 1.6805 - sparse_categorical_accuracy: 0.3317
Epoch 69/100
190/190 [==============================] - 2s 13ms/step - loss: 1.6793 - sparse_categorical_accuracy: 0.3337
Epoch 70/100
190/190 [==============================] - 2s 13ms/step - loss: 1.6809 - sparse_categorical_accuracy: 0.3316
Epoch 71/100
190/190 [==============================] - 2s 13ms/step - loss: 1.6788 - sparse_categorical_accuracy: 0.3372
Epoch 72/100
190/190 [==============================] - 2s 13ms/step - loss: 1.6775 - sparse_categorical_accuracy: 0.3363
Epoch 73/100
190/190 [==============================] - 2s 13ms/step - loss: 1.6806 - sparse_categorical_accuracy: 0.3348
Epoch 74/100
190/190 [===============

In [13]:
loss, acc = model.evaluate(test)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

43/43 [==============================] - 0s 4ms/step - loss: 2.2954 - sparse_categorical_accuracy: 0.1061

Test accuracy: 10.6%


Accuracy is very bad at only 10.6%. I will try to improve it with further tests.

## Test 2 (Validation)
In this test I use a validation set of data to increase accuracy

In [8]:
dataset = tfds.builder('eurosat')
info = dataset.info

In [9]:
print('For this test I take 10% away from training and add it to ')
(train, val, test) = tfds.load("eurosat/rgb", split=["train[:80%]", "train[80%:90%]", "train[90%:]"])
print('Train:', len(train), 'Validation:', len(val), 'Test:', len(test))

For this test I take 10% away from training and add it to 
Train: 21600 Validation: 2700 Test: 2700


In [10]:
train = train.batch(128)
train = train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train = train.cache() # stores training set in cache so subsequent calls are faster
train = train.shuffle(len(train)) # shuffles set

val = val.batch(128)
val = val.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

# do same to test set
test = test.batch(128)
test = test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test = test.cache()

In [11]:
model = Sequential([
  Flatten(input_shape=(64, 64, 3)),
  Dense(128, activation='relu'),
  Dense(info.features["label"].num_classes, activation='softmax')
])

# use default params for model compiler
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=[SparseCategoricalAccuracy()]
)

print("Adding a validation set.")
model.fit(
    train,
    epochs=100,
    validation_data=val,
)

Adding a validation set.
Epoch 1/100


ValueError: in user code:

    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1084, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\optimizers\optimizer.py", line 544, in minimize
        self.apply_gradient

In [19]:
loss, acc = model.evaluate(test)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

22/22 [==============================] - 0s 4ms/step - loss: 1.6541 - sparse_categorical_accuracy: 0.3956

Test accuracy: 39.6%


Accuracy is much better than last time, but still not so good.

## Test 3 (Optimizer)
This time I will try adding an optimizer while compiling the model

In [12]:
dataset = tfds.builder('eurosat')
info = dataset.info

In [13]:
(train, val, test) = tfds.load("eurosat/rgb", split=["train[:80%]", "train[80%:90%]", "train[90%:]"])
print('Train:', len(train), 'Validation', len(val), 'Test:', len(test))

Train: 21600 Validation 2700 Test: 2700


In [14]:
train = train.batch(128)
train = train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train = train.cache()
train = train.shuffle(len(train))

val = val.batch(128)
val = val.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

test = test.batch(128)
test = test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test = test.cache()

In [15]:
model = Sequential([
  Flatten(input_shape=(64, 64, 3)),
  Dense(128, activation='relu'),
  Dense(info.features["label"].num_classes, activation='softmax')
])

print("Using adam optimizer")
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=[SparseCategoricalAccuracy()]
)

model.fit(
    train,
    epochs=100,
    validation_data=val,
)

Using adam optimizer
Epoch 1/100


ValueError: in user code:

    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1084, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\optimizers\optimizer.py", line 544, in minimize
        self.apply_gradient

In [24]:
pred = model.predict(test, batch_size=128)

22/22 [==============================] - 0s 5ms/step


In [25]:
loss, acc = model.evaluate(test)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

22/22 [==============================] - 0s 4ms/step - loss: 1.3560 - sparse_categorical_accuracy: 0.5544

Test accuracy: 55.4%


Accuracy increased again from about 40% to 55%.

## Test 4 (More layers)
More layers makes the model more complex and helps it learn faster.

In [16]:
dataset = tfds.builder('eurosat')
info = dataset.info

In [17]:
(train, val, test) = tfds.load("eurosat/rgb", split=["train[:80%]", "train[80%:90%]", "train[90%:]"])
print('Train:', len(train), 'Validation', len(val), 'Test:', len(test))

Train: 21600 Validation 2700 Test: 2700


In [18]:
train = train.batch(128)
train = train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train = train.cache() # stores training set in cache so subsequent calls are faster
train = train.shuffle(len(train)) # shuffles set

val = val.batch(128)
val = val.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

# do same to test set
test = test.batch(128)
test = test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test = test.cache()

In [19]:
print("Adding a Conv2D layer and a MaxPooling2D layer.")
model = Sequential([
    Conv2D(input_shape=(64, 64, 3), filters=8, kernel_size=(3, 3), strides=1, activation='relu', padding='SAME'),
    MaxPooling2D(pool_size=(8, 8)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(info.features["label"].num_classes, activation='softmax')
])

# use default params for model compiler
model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=[SparseCategoricalAccuracy()]
)

model.fit(
    train,
    epochs=100,
    validation_data=val,
)

Adding a Conv2D layer and a MaxPooling2D layer.
Epoch 1/100


ValueError: in user code:

    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1084, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\dargu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\optimizers\optimizer.py", line 544, in minimize
        self.apply_gradient

In [32]:
loss, acc = model.evaluate(test)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

22/22 [==============================] - 0s 12ms/step - loss: 0.6216 - sparse_categorical_accuracy: 0.7900

Test accuracy: 79.0%


Accuracy increased to 79%. It's gotten a lot closer since 10% at test 1, but not as good as I was hoping for.

Unfortunately when I check over the notebook I get errors whenever I run model.fit. It was working before, but it isn't now and I don't know why. I no longer have the time to figure it out either as it is about 11:51 pm.
All I did was delete comments and add print statements and now it doesn't work.